In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from tkinter import *
import sqlite3

dataset = pd.read_csv('Restaurant_reviews.csv',
					delimiter='\t', quoting=3)
corpus = []
rras_code = "Wyd^H3R"
food_rev = {}
food_perc = {}

conn = sqlite3.connect('Restaurant_food_data.db')
c = conn.cursor()

for i in range(0, 1000):
	review = re.sub('[^a-zA-Z]', ' ', dataset['Review'][i])
	review = review.lower()
	review = review.split()

	ps = PorterStemmer()
	all_stopwords = stopwords.words('english')
	all_stopwords.remove('not')

	review = [ps.stem(word)
			for word in review if not word in set(all_stopwords)]
	
	review = ' '.join(review)
	corpus.append(review)

cv = CountVectorizer(max_features=1500)
X = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, -1].values

X_train, X_test, y_train, y_test = train_test_split(
	X, y, test_size=0.20, random_state=0)

classifier = GaussianNB()
classifier.fit(X_train, y_train)

y_pred = classifier.predict(X_test)

variables = []
clr_variables = []

foods = ["Idly", "Dosa", "Vada", "Roti", "Meals", "Veg Biryani",
		"Egg Biryani", "Chicken Biryani", "Mutton Biryani",
		"Ice Cream", "Noodles", "Manchooriya", "Orange juice",
		"Apple Juice", "Pineapple juice", "Banana juice"]

for i in foods:
	food_rev[i] = []
	food_perc[i] = [0.0, 0.0]


def init_data():

	conn = sqlite3.connect('Restaurant_food_data.db')
	c = conn.cursor()
	for i in range(len(foods)):
		c.execute("INSERT INTO item VALUES(:item_name,:no_of_customers,\
		:no_of_positives,:no_of_negatives,:pos_perc,:neg_perc)",
				{
					'item_name': foods[i],
					'no_of_customers': "0",
					'no_of_positives': "0",
					'no_of_negatives': "0",
					'pos_perc': "0.0%",
					'neg_perc': "0.0%"
				}
				)
	conn.commit()
	conn.close()


KeyError: 'Review'